## Processing Version 001 ECOSTRESS Data: Swath to Grid 

In this notebook, we convert the raw ECOSTRESS version 001 product to a grid.

In [57]:
!pip install pyresample

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 9.7 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 94.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.7/365.7 kB 111.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 122.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 112.8 MB/s eta 0:00:00


In [59]:
"""
Load the necessary packages and set environment variables
"""

# Import packages
import os, shutil, time, glob, warnings
import folium
import earthaccess
import pandas as pd
import geopandas as gpd
import rasterio as rio
import rioxarray as rxr
import h5py
import xarray as xr
import numpy as np
from matplotlib import pyplot as plt
from affine import Affine
from pyresample import geometry as geom

# Projection information
geog = 'EPSG:4326'  # Geographic projection
prj = 'EPSG:5070'  # Projected coordinate system- WGS 84 NAD83 UTM Zone 13N

# File path information
datadir = '/data-store/iplant/home/shared/esiil/HYR_SENSE/'

# File path information
print("Success")

ModuleNotFoundError: No module named 'pyresample'

In [47]:
ecodir = '/home/jovyan/HYR-SENSE/data/Drought-FireRisk/'

def list_files(path, ext, recursive):
    """
    List files of a specific type in a directory or subdirectories
    """
    if recursive is True:
        return glob.glob(os.path.join(path, '**', '*{}'.format(ext)), recursive=True)
    else:
        return glob.glob(os.path.join(path, '*{}'.format(ext)), recursive=False)

# Get a list of .nc files
nc_files = list_files(ecodir,"*.h5",recursive=True)
print(nc_files)

['/home/jovyan/HYR-SENSE/data/Drought-FireRisk/ECO2LSTE/ECOSTRESS_L2_LSTE_11158_004_20200624T121534_0601_01.h5', '/home/jovyan/HYR-SENSE/data/Drought-FireRisk/ECO4ESIPTJPL/ECOSTRESS_L4_ESI_PT-JPL_11158_004_20200624T121534_0601_01.h5', '/home/jovyan/HYR-SENSE/data/Drought-FireRisk/ECO1BGEO/ECOSTRESS_L1B_GEO_11158_004_20200624T121534_0601_01.h5']


In [48]:
# Function to open and print NetCDF items
def read_h5_file(fp):
    
    def print_attrs(name, obj):
        print(name)
        for key, val in obj.attrs.items():
            print(f"    {key}: {val}")
            
    with h5py.File(fp, 'r') as f:
        print(f'Contents of {fp}')
        f.visititems(print_attrs)

# Open the first file to examine the contents
read_h5_file(nc_files[2])

Contents of /home/jovyan/HYR-SENSE/data/Drought-FireRisk/ECO1BGEO/ECOSTRESS_L1B_GEO_11158_004_20200624T121534_0601_01.h5
Geolocation
    Projection: The latitude, longitude, and height are relative to the WGS-84
ellipsoid. Specifically the projection used is described by
the Well-Known Text (WKT):

GEOGCS["WGS 84",
    DATUM["WGS_1984",
        SPHEROID["WGS 84",6378137,298.257223563,
            AUTHORITY["EPSG","7030"]],
        AUTHORITY["EPSG","6326"]],
    PRIMEM["Greenwich",0],
    UNIT["degree",0.0174532925199433],
    AUTHORITY["EPSG","4326"]]

    Projection_WKT: GEOGCS["WGS 84",
    DATUM["WGS_1984",
        SPHEROID["WGS 84",6378137,298.257223563,
            AUTHORITY["EPSG","7030"]],
        AUTHORITY["EPSG","6326"]],
    PRIMEM["Greenwich",0],
    UNIT["degree",0.0174532925199433],
    AUTHORITY["EPSG","4326"]]

Geolocation/height
    Units: m
Geolocation/land_fraction
    Units: percentage
    fill: -9999
    valid_max: 100
    valid_min: 0
Geolocation/latitude
    Units

In [49]:
f = h5py.File(nc_files[0])             # Read in ECOSTRESS HDF5 file
ecoName = nc_files[0].split('.h5')[0]  # Keep original filename
print(ecoName)

/home/jovyan/HYR-SENSE/data/Drought-FireRisk/ECO2LSTE/ECOSTRESS_L2_LSTE_11158_004_20200624T121534_0601_01


In [50]:
# Create a list of all SDS inside of the .h5 file
eco_objs = []
f.visit(eco_objs.append)
ecoSDS = [str(obj) for obj in eco_objs if isinstance(f[obj], h5py.Dataset)] 
for dataset in ecoSDS[0:10]: 
    print(dataset)

L2 LSTE Metadata/AncillaryNWP
L2 LSTE Metadata/BandSpecification
L2 LSTE Metadata/CloudMaxTemperature
L2 LSTE Metadata/CloudMeanTemperature
L2 LSTE Metadata/CloudMinTemperature
L2 LSTE Metadata/CloudSDevTemperature
L2 LSTE Metadata/Emis1GoodAvg
L2 LSTE Metadata/Emis2GoodAvg
L2 LSTE Metadata/Emis3GoodAvg
L2 LSTE Metadata/Emis4GoodAvg


In [51]:
# Subset list to ETinst and ETinstUncertainty
sds = ['LST']
ecoSDS = [dataset for dataset in ecoSDS if dataset.endswith(tuple(sds))]
print(ecoSDS)
for dataset in ecoSDS:
    print(dataset.split('/')[-1])

['SDS/LST']
LST


In [52]:
# Find the matching ECO1BGEO file from the file list
geo = list_files(os.path.join(ecodir,'ECO1BGEO/'), ext="*.h5",recursive=True)
print(geo)

['/home/jovyan/HYR-SENSE/data/Drought-FireRisk/ECO1BGEO/ECOSTRESS_L1B_GEO_11158_004_20200624T121534_0601_01.h5']


In [54]:
# Open Geo File
g = h5py.File(geo[0])
geo_objs = []
g.visit(geo_objs.append)

# Search for lat/lon SDS inside data file
latSD = [str(obj) for obj in geo_objs if isinstance(g[obj], h5py.Dataset) and '/latitude' in obj]
lonSD = [str(obj) for obj in geo_objs if isinstance(g[obj], h5py.Dataset) and '/longitude' in obj]

# Open SDS as arrays
lat = g[latSD[0]][()].astype(float)
lon = g[lonSD[0]][()].astype(float)

# Read the array dimensions
dims = lat.shape
print(dims)

(5632, 5400)


In [55]:
# Set swath definition from lat/lon arrays
swathDef = geom.SwathDefinition(lons=lon, lats=lat)
swathDef.corners

NameError: name 'geom' is not defined

In [ ]:
# Define the lat/lon for the middle of the swath
mid = [int(lat.shape[1] / 2) - 1, int(lat.shape[0] / 2) - 1]
midLat, midLon = lat[mid[0]][mid[1]], lon[mid[0]][mid[1]]
midLat, midLon

In [ ]:
# Define AEQD projection centered at swath center
epsgConvert = pyproj.Proj("+proj=aeqd +lat_0={} +lon_0={}".format(midLat, midLon))

# Use info from AEQD projection bbox to calculate output cols/rows/pixel size
llLon, llLat = epsgConvert(np.min(lon), np.min(lat), inverse=False)
urLon, urLat = epsgConvert(np.max(lon), np.max(lat), inverse=False)
areaExtent = (llLon, llLat, urLon, urLat)
cols = int(round((areaExtent[2] - areaExtent[0]) / 70))  # 70 m pixel size
rows = int(round((areaExtent[3] - areaExtent[1]) / 70))

In [ ]:
# Define Geographic projection
epsg, proj, pName = '4326', 'longlat', 'Geographic'

# Define bounding box of swath
llLon, llLat, urLon, urLat = np.min(lon), np.min(lat), np.max(lon), np.max(lat)
areaExtent = (llLon, llLat, urLon, urLat)

# Create area definition with estimated number of columns and rows
projDict = pyproj.CRS("epsg:4326")
areaDef = geom.AreaDefinition(epsg, pName, proj, projDict, cols, rows, areaExtent)

In [ ]:
# Square pixels and calculate output cols/rows
ps = np.min([areaDef.pixel_size_x, areaDef.pixel_size_y])
cols = int(round((areaExtent[2] - areaExtent[0]) / ps))
rows = int(round((areaExtent[3] - areaExtent[1]) / ps))

# Set up a new Geographic area definition with the refined cols/rows
areaDef = geom.AreaDefinition(epsg, pName, proj, projDict, cols, rows, areaExtent)

In [ ]:
# Get arrays with information about the nearest neighbor to each grid point 
index, outdex, indexArr, distArr = kdt.get_neighbour_info(swathDef, areaDef, 210, neighbours=1)